## Constants

In [1]:
import sys, os
import pandas as pd
import numpy as np
import subprocess
import gc
import optuna
from datetime import datetime, timezone
import warnings
import xgboost as xgb
import joblib as jl
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import matthews_corrcoef
from mlflow.models import infer_signature
import mlflow
import random
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

today = datetime.now(timezone.utc).strftime("%Y_%m_%d")
warnings.filterwarnings("ignore")
from hyper_params import (
    mushroom_tuning_2024_08_06_1722934727_params,
)

# helpers
sys.path.append("..")
from helpers.loss_functions import *
from helpers.mlflow import *


SEED = 108
random.seed(SEED)
N_FOLDS = 5
# data
train_path = "../data/mushrooms/train.csv"
test_path = "../data/mushrooms/test.csv"
cache_path = "../data/mushrooms/cache"
# model
is_tunning = True
try:
    rs = subprocess.check_output("nvidia-smi")
    device = "cuda" if rs is not None else "cpu"
except (
    Exception
):  # this command not being found can raise quite a few different errors depending on the configuration
    print("No Nvidia GPU in system!")
    device = "cpu"
goal = "binary:logistic"

# custom metric
objective_dict = {
    "binary:logistic": {
        "metric": {
            "is_custom": False,
            "name": "logloss",
            "fval": None,
        },
        "direction": "minimize",
    }
}
# objective_dict = {
#     "binary:logistic": {
#         "metric": {
#             "is_custom": True,
#             "name": "MCC",
#             "fval": mcc_metric_v2,
#         },
#         "direction": "maximize",
#     }
# }
metric = objective_dict[goal]["metric"]["name"]
is_custom_metric = objective_dict[goal]["metric"]["is_custom"]
fval = objective_dict[goal]["metric"]["fval"]
direction = objective_dict[goal]["direction"]
best_params = {
    "device": device,
    "verbosity": 0,
    "objective": goal,
    # "eval_metric": fval,
}
best_params.update(mushroom_tuning_2024_08_06_1722934727_params)
best_params

{'device': 'cuda',
 'verbosity': 0,
 'objective': 'binary:logistic',
 'tree_method': 'hist',
 'eta': 0.0696294726051571,
 'max_depth': 0,
 'min_child_weight': 1,
 'gamma': 0.044230646284796976,
 'subsample': 0.9405269471473167,
 'colsample_bytree': 0.2999355523666192,
 'lambda': 0.9746051811186938,
 'alpha': 4.210861941737071}

## Prepare data

In [2]:
train = pd.read_csv(train_path)
print(f"train size: {train.shape}")
test = pd.read_csv(test_path)
print(f"test size: {test.shape}")

train size: (3116945, 22)
test size: (2077964, 21)


In [3]:
target = "class"

categorical_cols = (
    train.drop(columns=target).select_dtypes(include="object").columns.to_list()
)
for c in categorical_cols:
    train[c] = train[c].astype("category")
    test[c] = test[c].astype("category")
numerical_cols = (
    train.drop(columns="id").select_dtypes(include="number").columns.to_list()
)

In [4]:
# X_train, X_val, y_train, y_val = train_test_split(
#     train.drop(columns=target),
#     train[target],
#     test_size=0.2,
#     random_state=42,
#     stratify=train[target],
# )
X_test = test
X_train = train.drop(columns=target)
y_train = train[target]
gc.collect()

132

## Data preprocessing

In [5]:
from sklearn.preprocessing import MinMaxScaler

# Create the numerical and categorical pipelines
numerical_pipeline = Pipeline(
    steps=[
        ("num_imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
        ("minmax", MinMaxScaler()),
    ]
)

categorical_pipeline = Pipeline(
    steps=[
        ("cat_imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

# Combine the pipelines into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, numerical_cols),
        ("cat", categorical_pipeline, categorical_cols),
    ]
)

# Create the full pipeline with the XGBoost model
data_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
    ]
)

# Preprocess the data
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)
# X_val_transformed = preprocessor.transform(X_val)
# Binarize the target labels
lb = LabelBinarizer()

y_train_binarized = lb.fit_transform(y_train)
# y_val_binarized = lb.transform(y_val)

# prepare data for training
dtrain = xgb.DMatrix(X_train_transformed, label=y_train_binarized)
# dval = xgb.DMatrix(X_val_transformed, label=y_val_binarized)
dtest = xgb.DMatrix(X_test_transformed)

gc.collect()

62

## CV

In [6]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

clf: xgb.XGBClassifier = xgb.XGBClassifier(
    **best_params,
    # callbacks=[xgb.callback.EarlyStopping(50)],
    n_estimators=200,
)

In [7]:
from tqdm import tqdm
kf = KFold(n_splits=N_FOLDS)
y_preds = []
y_trues = []
for train_index, test_index in tqdm(kf.split(X_train_transformed, y_train_binarized)):
    X_train, X_test = X_train_transformed[train_index], X_train_transformed[test_index]
    y_train, y_test = y_train_binarized[train_index], y_train_binarized[test_index]
    clf.fit(
        X=X_train,
        y=y_train,
        eval_set=[(X_test, y_test)],
    )

    y_pred = clf.predict(X_test)
    y_preds.append(y_pred)
    y_trues.append(y_test)
# Concatenate the predictions and true labels
y_preds_concat = np.concatenate(y_preds)
y_trues_concat = np.concatenate(y_trues)
mcc = matthews_corrcoef(y_trues_concat, y_preds_concat)
print(f"Validation mcc score: {mcc}")

0it [00:00, ?it/s]

[0]	validation_0-logloss:0.65986
[1]	validation_0-logloss:0.61860
[2]	validation_0-logloss:0.56700
[3]	validation_0-logloss:0.52695
[4]	validation_0-logloss:0.49723
[5]	validation_0-logloss:0.46795
[6]	validation_0-logloss:0.43857
[7]	validation_0-logloss:0.40853
[8]	validation_0-logloss:0.37959
[9]	validation_0-logloss:0.35536
[10]	validation_0-logloss:0.33101
[11]	validation_0-logloss:0.31189
[12]	validation_0-logloss:0.29768
[13]	validation_0-logloss:0.28191
[14]	validation_0-logloss:0.26937
[15]	validation_0-logloss:0.25276
[16]	validation_0-logloss:0.24044
[17]	validation_0-logloss:0.22796
[18]	validation_0-logloss:0.21722
[19]	validation_0-logloss:0.20703
[20]	validation_0-logloss:0.19884
[21]	validation_0-logloss:0.18892
[22]	validation_0-logloss:0.17909
[23]	validation_0-logloss:0.16953
[24]	validation_0-logloss:0.16246
[25]	validation_0-logloss:0.15400
[26]	validation_0-logloss:0.15024
[27]	validation_0-logloss:0.14461
[28]	validation_0-logloss:0.13728
[29]	validation_0-loglos

1it [00:14, 14.82s/it]

[0]	validation_0-logloss:0.66003
[1]	validation_0-logloss:0.61873
[2]	validation_0-logloss:0.56713
[3]	validation_0-logloss:0.52712
[4]	validation_0-logloss:0.49732
[5]	validation_0-logloss:0.46804
[6]	validation_0-logloss:0.43873
[7]	validation_0-logloss:0.40866
[8]	validation_0-logloss:0.37969
[9]	validation_0-logloss:0.35544
[10]	validation_0-logloss:0.33111
[11]	validation_0-logloss:0.31199
[12]	validation_0-logloss:0.29779
[13]	validation_0-logloss:0.28200
[14]	validation_0-logloss:0.26948
[15]	validation_0-logloss:0.25288
[16]	validation_0-logloss:0.24056
[17]	validation_0-logloss:0.22806
[18]	validation_0-logloss:0.21733
[19]	validation_0-logloss:0.20712
[20]	validation_0-logloss:0.19890
[21]	validation_0-logloss:0.18899
[22]	validation_0-logloss:0.17917
[23]	validation_0-logloss:0.16960
[24]	validation_0-logloss:0.16252
[25]	validation_0-logloss:0.15403
[26]	validation_0-logloss:0.15025
[27]	validation_0-logloss:0.14504
[28]	validation_0-logloss:0.13767
[29]	validation_0-loglos

2it [00:29, 14.72s/it]

...

[0]	validation_0-logloss:0.66012
[1]	validation_0-logloss:0.61895
[2]	validation_0-logloss:0.56733
[3]	validation_0-logloss:0.52728
[4]	validation_0-logloss:0.49750
[5]	validation_0-logloss:0.46823
[6]	validation_0-logloss:0.43890
[7]	validation_0-logloss:0.40887
[8]	validation_0-logloss:0.37986
[9]	validation_0-logloss:0.35563
[10]	validation_0-logloss:0.33128
[11]	validation_0-logloss:0.31215
[12]	validation_0-logloss:0.29794
[13]	validation_0-logloss:0.28425
[14]	validation_0-logloss:0.27160
[15]	validation_0-logloss:0.25483
[16]	validation_0-logloss:0.24240
[17]	validation_0-logloss:0.22977
[18]	validation_0-logloss:0.21891
[19]	validation_0-logloss:0.20862
[20]	validation_0-logloss:0.20034
[21]	validation_0-logloss:0.19034
[22]	validation_0-logloss:0.18044
[23]	validation_0-logloss:0.17079
[24]	validation_0-logloss:0.16365
[25]	validation_0-logloss:0.15511
[26]	validation_0-logloss:0.15131
[27]	validation_0-logloss:0.14563
[28]	validation_0-logloss:0.13822
[29]	validation_0-loglos

4it [00:58, 14.64s/it]

[0]	validation_0-logloss:0.65997
[1]	validation_0-logloss:0.61873
[2]	validation_0-logloss:0.56707
[3]	validation_0-logloss:0.52705
[4]	validation_0-logloss:0.49729
[5]	validation_0-logloss:0.46799
[6]	validation_0-logloss:0.43868
[7]	validation_0-logloss:0.40862
[8]	validation_0-logloss:0.37963
[9]	validation_0-logloss:0.35538
[10]	validation_0-logloss:0.33101
[11]	validation_0-logloss:0.31186
[12]	validation_0-logloss:0.29766
[13]	validation_0-logloss:0.28187
[14]	validation_0-logloss:0.26933
[15]	validation_0-logloss:0.25272
[16]	validation_0-logloss:0.24039
[17]	validation_0-logloss:0.22788
[18]	validation_0-logloss:0.21712
[19]	validation_0-logloss:0.20691
[20]	validation_0-logloss:0.19870
[21]	validation_0-logloss:0.18878
[22]	validation_0-logloss:0.17894
[23]	validation_0-logloss:0.16937
[24]	validation_0-logloss:0.16229
[25]	validation_0-logloss:0.15379
[26]	validation_0-logloss:0.15001
[27]	validation_0-logloss:0.14438
[28]	validation_0-logloss:0.13705
[29]	validation_0-loglos

5it [01:13, 14.76s/it]


Validation mcc score: 0.9834362827787043


In [11]:
viz_feature_important(clf=clf)

NameError: name 'viz_feature_important' is not defined

In [8]:
submit_df = pd.DataFrame()
submit_df["id"] = test["id"]
y_preds = clf.predict(X_test_transformed)
pred_classes = lb.inverse_transform(y_preds)
submit_df["class"] = pred_classes
submit_df.to_csv("submission.csv", index=False)